<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 3 Assignment 1*

# Recurrent Neural Networks and Long Short Term Memory (LSTM)

![Monkey at a typewriter](https://upload.wikimedia.org/wikipedia/commons/thumb/3/3c/Chimpanzee_seated_at_typewriter.jpg/603px-Chimpanzee_seated_at_typewriter.jpg)

It is said that [infinite monkeys typing for an infinite amount of time](https://en.wikipedia.org/wiki/Infinite_monkey_theorem) will eventually type, among other things, the complete works of Wiliam Shakespeare. Let's see if we can get there a bit faster, with the power of Recurrent Neural Networks and LSTM.

This text file contains the complete works of Shakespeare: https://www.gutenberg.org/files/100/100-0.txt

Use it as training data for an RNN - you can keep it simple and train character level, and that is suggested as an initial approach.

Then, use that trained RNN to generate Shakespearean-ish text. Your goal - a function that can take, as an argument, the size of text (e.g. number of characters or lines) to generate, and returns generated text of that size.

Note - Shakespeare wrote an awful lot. It's OK, especially initially, to sample/use smaller data and parameters, so you can have a tighter feedback loop when you're trying to get things running. Then, once you've got a proof of concept - start pushing it more!

In [ ]:
# TODO - Words, words, mere words, no matter from the heart.

In [1]:
import pandas as pd
import numpy as np
import random
import sys
import os
import tensorflow
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.optimizers import RMSprop

In [12]:
df = pd.read_csv('sonnets_csv.csv', encoding='latin-1')
df.head(20)

,1
0,"From fairest creatures we desire increase,"
1,"That thereby beautyÕs rose might never die,"
2,"But as the riper should by time decease,"
3,His tender heir might bear his memory:
4,"But thou contracted to thine own bright eyes,"
5,FeedÕst thy lightÕs flame with self-substantia...
6,"Making a famine where abundance lies,"
7,"Thy self thy foe, to thy sweet self too cruel:"
8,"Thou that art now the worldÕs fresh ornament,"
9,"And only herald to the gaudy spring,"


In [4]:
df.shape

(2307, 1)

In [13]:
df_list = df['1'].tolist()
df_list

['From fairest creatures we desire increase,',
 'That thereby beautyÕs rose might never die,',
 'But as the riper should by time decease,',
 'His tender heir might bear his memory:',
 'But thou contracted to thine own bright eyes,',
 'FeedÕst thy lightÕs flame with self-substantial fuel,',
 'Making a famine where abundance lies,',
 'Thy self thy foe, to thy sweet self too cruel:',
 'Thou that art now the worldÕs fresh ornament,',
 'And only herald to the gaudy spring,',
 'Within thine own bud buriest thy content,',
 'And, tender churl, makÕst waste in niggarding:',
 'Pity the world, or else this glutton be,',
 'To eat the worldÕs due, by the grave and thee.',
 '2',
 'When forty winters shall besiege thy brow,',
 'And dig deep trenches in thy beautyÕs field,',
 'Thy youthÕs proud livery so gazed on now,',
 'Will be a tattered weed of small worth held:',
 'Then being asked, where all thy beauty lies,',
 'Where all the treasure of thy lusty days;',
 'To say, within thine own deep sunken e

In [14]:
# Encode Data as Chars
text = " ".join(df_list)
chars = list(set(text))

char_int = {c:i for i,c in enumerate(chars)}
int_char = {i:c for i,c in enumerate(chars)}

In [15]:
# Create Sequence 
# The lists are: encoded (chars encoded as integers), sequences (40 chars per entry), next_chars ()
maxlen = 40
step = 5

encoded = [char_int[c] for c in text]

sequences = [] # Each element is 40 chars long
next_chars = [] # One element for each sequence for prediction

for i in range(0, len(encoded) - maxlen, step):
    sequences.append(encoded[i : i + maxlen])
    next_chars.append(encoded[i + maxlen])
    
print('sequences', len(sequences))

sequences 18826


In [17]:
# Initiali
x = np.zeros((len(sequences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sequences), len(chars)), dtype=np.bool)

for i, sequence in enumerate(sequences):
    for t, char in enumerate(sequence):
        x[i,t,char] = 1
        
    y[i, next_chars[i]] = 1

In [18]:
# build LSTM model
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars)))) #input shape of a singular observ.
model.add(Dense(len(chars), activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [19]:
def sample(preds):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / 1
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [20]:
def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_int[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = int_char[next_index]

            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [ ]:
model.fit(x, y,
          batch_size=128,
          epochs=20,
          callbacks=[print_callback])

Train on 18826 samples
Epoch 1/20
18816/18826 [============================>.] - ETA: 0s - loss: 3.1361
----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "r white despair: A third nor red, nor wh"
r white despair: A third nor red, nor wh   o   o oo       o e e   s  o  o  e  t  o   o  e  o o      e   a   e      o s oo o oo t      e    a  t  s t  o  so oo    e o         o     o e        e t es    e    o    o  t   s  o t e  o   ee o   n   o        or        oi  e  oa t        oo  e      oo e  a  o   s   o        o     os  t  ou  s o       o  e o     s   o   e e    e   n o    e     o o     o o  e    e  s o     s       o  e    e  s  o
----- diversity: 0.5
----- Generating with seed: "r white despair: A third nor red, nor wh"
r white despair: A third nor red, nor wht aori a eei  um a eioa stss n  hd a e imooe  ul le  r vh t ie i it  eeoo he  s so o cor h ro he t , t aasoeae o  nrnooos ir   ee ,ounnh,   e  stoe tdodo A  I  o s a  os ciiveotee  nlint  t ieoo

# Resources and Stretch Goals

## Stretch goals:
- Refine the training and generation of text to be able to ask for different genres/styles of Shakespearean text (e.g. plays versus sonnets)
- Train a classification model that takes text and returns which work of Shakespeare it is most likely to be from
- Make it more performant! Many possible routes here - lean on Keras, optimize the code, and/or use more resources (AWS, etc.)
- Revisit the news example from class, and improve it - use categories or tags to refine the model/generation, or train a news classifier
- Run on bigger, better data

## Resources:
- [The Unreasonable Effectiveness of Recurrent Neural Networks](https://karpathy.github.io/2015/05/21/rnn-effectiveness/) - a seminal writeup demonstrating a simple but effective character-level NLP RNN
- [Simple NumPy implementation of RNN](https://github.com/JY-Yoon/RNN-Implementation-using-NumPy/blob/master/RNN%20Implementation%20using%20NumPy.ipynb) - Python 3 version of the code from "Unreasonable Effectiveness"
- [TensorFlow RNN Tutorial](https://github.com/tensorflow/models/tree/master/tutorials/rnn) - code for training a RNN on the Penn Tree Bank language dataset
- [4 part tutorial on RNN](http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/) - relates RNN to the vanishing gradient problem, and provides example implementation
- [RNN training tips and tricks](https://github.com/karpathy/char-rnn#tips-and-tricks) - some rules of thumb for parameterizing and training your RNN